In [16]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr_asc
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [27]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_wesad = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [
        dr_w.Phases.BASE,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    # [
    #     dr_w.Phases.BASE,
    # ]
]

wesad_label_type = "stai"
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 0.8


In [29]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_wesad in enumerate(model_phases_wesad):
    print(f"WESAD PHASES {j} " + "-"*50)
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
    # drop subjects with noisy data
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0)
    y_a = y_a.drop(labels=inds, axis=0)
    
    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]
    
    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    # print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
    # print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

    # 0-1 scaling
    for c in range(3, len(x_a.columns)):
        data_col = x_a[x_a.columns[c]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x_a[x_a.columns[c]] = data_col
    # 0-1 scaling
    for c in range(3, len(x_b.columns)):
        data_col = x_b[x_b.columns[c]]
        data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
        x_b[x_b.columns[c]] = data_col

    acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
    }
    reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
    }
    num_iters = 10
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0.55:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
    print("\n")

WESAD PHASES 0 --------------------------------------------------
y_train:
1    911
0    651
Name: label, dtype: int64
y_test:
0    35
1    20
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [12 43]
Model LogReg, Predictions: [0 1], [54  1]
Model XGB, Predictions: [0 1], [22 33]
y_train:
1    911
0    651
Name: label, dtype: int64
y_test:
0    35
1    20
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 7 48]
Model LogReg, Predictions: [0 1], [54  1]
Model XGB, Predictions: [0 1], [43 12]
y_train:
1    911
0    651
Name: label, dtype: int64
y_test:
0    35
1    20
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [11 44]
Model LogReg, Predictions: [0 1], [54  1]
Model XGB, Predictions: [0 1], [31 24]
y_train:
1    910
0    652
Name: label, dtype: int64
y_test:
0    34
1    21
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 9 46]
Model LogReg, Predictions: [0], [55]
Model XGB, Predictions: [0 1], [34 21]
y_train:
1    912
0    650
Name: label, dtype: 

In [ ]:
# TRAIN ON WESAD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_wesad in enumerate(model_phases_wesad):
        print(f"WESAD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
        x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
        # drop subjects with noisy data
        inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
        x_b = x_b.drop(labels=inds, axis=0)
        y_b = y_b.drop(labels=inds, axis=0)
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500
        y_b["subject"] = y_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
        print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

        acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.6:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")